# Marketing Automation Project

If you want to try the tool without delving into the technical stuff, visit our frontend website:

https://smark-frontend.vercel.app/

## Goal

We want to build a tool to assist startups and small businesses in crafting business and marketing plans, as well as generating compelling marketing material.

We harness the power of GPT-3.5 and GPT-4 for content generation.
They are available via OpenAI APIs.

https://platform.openai.com/docs/models/overview

Some of the features we intended to include:
- Business Plan Generation
- Marketing Plan Generation
- Website Generation
- Products' Landing Page Generation
- Blog Articles Generation
- Social Media Post Generation
- Social Media Ads Generation
- Email Marketing Generation

## Table of Contents

1. [OpenAI API](#openai)
2. [FastAPI (+Uvicorn)](#fastapi)
3. [AWS App Runner](#apprunner)
4. [NextJS](#nextjs)
5. [Social Management](#social)
6. [Email Management](#email)

<a id='openai'></a>
### OpenAI API

All the content for our tool is generated using OpenAI models, which power the 

First thing, we need to install openAI and get an API key.

https://platform.openai.com/account/api-keys

If you want to follow our architecture you also need the dotenv package as we will use a .env file to store our api keys in environmental variables.

In [1]:
!pip install openai python-dotenv

from dotenv import load_dotenv
load_dotenv()

import openai
import os
openai.api_key = os.environ.get("OPEN_AI_API_KEY")


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Let's now try to send a request.

In [2]:
prompt = """
Which country has the best food in the world?
"""

result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", # this is the model that powers ChatGPT
    messages=[{"role": "user", "content": prompt}],
)
result

<OpenAIObject chat.completion id=chatcmpl-7kwgUiWzfG75VPUYkiluXSYyfSBmE at 0x295ca563e50> JSON: {
  "id": "chatcmpl-7kwgUiWzfG75VPUYkiluXSYyfSBmE",
  "object": "chat.completion",
  "created": 1691423026,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "It is subjective to determine which country has the best food in the world, as personal preferences vary. However, several countries are renowned for their culinary traditions and are often mentioned as having exceptional food. These include Italy for its pasta and pizza, France for its gourmet cuisine, Japan for its sushi and sashimi, India for its diverse and flavorful dishes, and Mexico for its vibrant and spicy cuisine. Ultimately, the best food is a matter of individual taste and exploration."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 18,
    "completion_tokens": 94,
    "total_tokens": 112
  }
}

While the answer from the model may vary, we all know the country with the best food in the world is Italy :P

Anyway to use the model output further in our code we need to be more specific in our prompt and ask for a structured output.

In [3]:
prompt = """
Which country has the best food in the world?
Write only a json in your response with key 'country' and value the country chosen.
"""

result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", # this is the model that powers ChatGPT
    messages=[{"role": "user", "content": prompt}],
)
result

<OpenAIObject chat.completion id=chatcmpl-7kwgZtaGfXiXRrJY7Qzi1u2peVFbH at 0x295ea0e2810> JSON: {
  "id": "chatcmpl-7kwgZtaGfXiXRrJY7Qzi1u2peVFbH",
  "object": "chat.completion",
  "created": 1691423031,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"country\": \"Italy\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 36,
    "completion_tokens": 9,
    "total_tokens": 45
  }
}

Now we can access the structured response and use/store it.

The answer is always a string so we can use the 'eval' function to tell python to interpret the string as code and give us a dictionary!

In [4]:
answer = eval(result["choices"][0]["message"]["content"])
answer

{'country': 'Italy'}

This "trick" is actually the backbone of the whole project.

We can store a multitude of prompts to be used and retrieve the generated content from the OpenAI response.

As an example let's see how we can generate a list of suggested business names.

In [5]:
project_root = os.path.dirname(os.getcwd())
PROMPT_FOLDER = os.path.join(project_root, 'prompt')

# we need to define some variables to be used in the prompt
language = "english"
business_description = """
My name is George Cannels and I've always had a green thumb. 
I have always had a vegetable garden and the people of the village come to me to get fruit, vegetables or seedlings. 
Since I'm retired now, I'd like to cultivate this passion of mine more and give the opportunity to let as many people 
as possible taste the products from my garden, as well as teaching others who can carry on my passion.
"""

with open(os.path.join(PROMPT_FOLDER, 'generate_business_names'), 'r') as file:

    # prompt is stored as a f-string
    prompt = eval(file.read())

result = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
)
names = eval(result["choices"][0]["message"]["content"])
names["business_names"]

['Garden Delights',
 'The Veggie Patch',
 'Nurturing Nature',
 'Harvest Haven',
 'Sow and Grow',
 'Green Thumb Gardens',
 'Bounty From Earth',
 'The Seedling Sanctuary',
 'Food for Thought',
 'Cultivating Connections']

<a id='fastapi'></a>
### FastAPI (+Uvicorn)

FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.7+.

https://fastapi.tiangolo.com/

Uvicorn is an ASGI web server implementation for Python.

https://www.uvicorn.org/

Let's start by installing the required packages.

In [6]:
!pip install fastapi
import fastapi


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Creating an app and its endpoints is super easy with FastAPI.

In [7]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"Hello": "World"}

We can run the Uvicorn webserver using the command: <code>uvicorn main:app</code> in our terminal.

For development is possible to add the --reload command, which reloads automatically the server whenever we make changes to the code.

<code>uvicorn main:app --reload</code>

Uvicorn default settings are host: 127.0.0.1 and port 8000. We can change those passing --host and --port parameters.

We can also specify the number of worker processes passing the parameter --workers.

So for a production environment the command will look something like this:

<code>uvicorn main:app --host 0.0.0.0 --port 80 --workers 10</code>

<a id='apprunner'></a>
### AWS App Runner

We have a running instance of our web server on AWS App Runner.

https://aws.amazon.com/apprunner/

App Runner allows to build, deploy and run web applications and API services.

It will build a containerized application directly from our source code and every time we push new code it will rebuild our application.

Alternatively you can upload your container image on Amazon ECR (Elastic Container Registry) and build the application from your image.

You can also create a configuration file (apprunner.yaml) where you can specify various settings:
- Runtime and version
- Network configuration
- Build commands (like pip install -r requirements.txt for installing your required packages)
- Run commands (in our case we launch uvicorn as shown before)
- Secrets (app secrets are loaded as environment variables that we can access in our code). These are NOT to be saved in plain text in the yaml file, but instead should be saved and loaded using AWS Secret Manager)

Let's now send a request to our webserver!

In [8]:
!pip install requests
import requests


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
url = "https://cmamp322z2.eu-central-1.awsapprunner.com"
response = requests.get(url)
response.json()

{'Hello': 'World'}

In [10]:
# Let's generate some business name suggestions!
url = "https://cmamp322z2.eu-central-1.awsapprunner.com/generate_business_names_api/3535"
response = requests.post(url)
data = response.json()

request_queue_id = data["request_queue_id"]
data

{'request_queue_id': 321}

Every I/O bound request is inserted in a request queue table.
We can check the results of our request accessing the table.

This would be much faster using a Redis cache, but to keep a simpler infrastructure we are just using another PostgreSQL table in our existing db.

In [28]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from db.db_manager import get_df_from_table

# helper function that queries the database and returns a pandas DataFrame
df = get_df_from_table("request_queue").sort_values(by="id", ascending=False)
df

,id,request_url,status,exception,response
318,321,http://cmamp322z2.eu-central-1.awsapprunner.co...,COMPLETE,None,"{'business_names': ['Garden Delights', 'Green ..."
317,320,http://cmamp322z2.eu-central-1.awsapprunner.co...,COMPLETE,None,"{'business_names': ['Garden Delights', 'Fresh ..."
316,319,http://cmamp322z2.eu-central-1.awsapprunner.co...,COMPLETE,None,"{'business_names': ['Garden Delights', 'Green ..."
315,318,http://cmamp322z2.eu-central-1.awsapprunner.co...,COMPLETE,None,{'target_market': 'Il mercato di riferimento d...
314,317,http://cmamp322z2.eu-central-1.awsapprunner.co...,COMPLETE,None,"{'founder_info': 'Sono Matteo Tegnenti, fondat..."
...,...,...,...,...,...
4,7,/,IN_PROGRESS,None,None
3,6,/,IN_PROGRESS,None,None
2,5,/,IN_PROGRESS,None,None
1,2,/,IN_PROGRESS,None,None


In [29]:
# Here the business names suggested
business_names = df.loc[df["id"]==request_queue_id, "response"].iloc[0]

In [30]:
business_names

{'business_names': ['Garden Delights',
  'Green Thumb Produce',
  'The Village Garden',
  "Nature's Bounty",
  'Seedlings & More',
  'Fresh Harvest',
  'The Veggie Patch',
  'Farm to Fork',
  'Nurturing Nature',
  'Garden Gurus']}

You can check all the available endpoints at this url:

https://cmamp322z2.eu-central-1.awsapprunner.com/docs

<a id='nextjs'></a>
### Next.js

Next.js is a React framework for building full-stack web applications. 
It also has improved support for TypeScript, with better type checking and more efficient compilation.

https://nextjs.org/

Vercel is a cloud platform that helps you develop, preview, and ship web applications with React and other frontend frameworks. It offers features such as serverless functions, global edge network, automatic previews, and more. It is also the creator of NextJs thus allow an extremely fast deploy experience for simple projects like this one.

https://vercel.com/


We created a simple frontend that allow us to connect with the backend and generate all kind of content we need through openAI APIs.

The frontend structure is composed of a main home page, where it is possible look to all business ideas. A business page that allows to generate the sections of the selecte business.

It is also possible to sign in with a Gmail account to save the generated content, and retrieve it aftewards.

![frontend1](../images/frontend_screenshot.png)

![frontend2](../images/frontend_screenshot2.png)

![frontend3](../images/frontend_screenshot3.png)

Try it yourself at https://smark-frontend.vercel.app !

<a id='social'></a>
### Social Management

We also created a Social management module (currently not available on our frontend) to be able to programmatically post, retrieve engagement metrics and manage user comments!

The idea is to be able to automatize social media management and customer support both reducing the cost and time expenditures for the business, while also improving the customer support quality for the business.

This last part is debatable, but with the correct context a customer support bot powered by OpenAI models is super helpful. Nowhere near the chatbots we are currently finding on most websites. We believe it can almost be at par with a human if provided all relevant context information. Without the possible human errors.

The challenging part is providing the context, and allowing for the context to grow after experiencing different customer queries/requests.

Anyway back to our technical stuff.

We'll provide code examples for Facebook, but the same can be replicated for any social that provides APIs.

#### Post
To post on Facebook, and for that matter any other operation, we would need a Page Token.

You can read more about OAuth2.0 and obtaining a token here:
- https://developers.facebook.com/docs/facebook-login/guides/advanced/manual-flow/
- https://developers.facebook.com/docs/pages/access-tokens/

Posting then is as easy as it can get!
Retrieving engagement is as trivial so we'll skip the explanation.

You can find all facebook API reference here:

https://developers.facebook.com/docs/graph-api/reference/

In [13]:
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

page_id = os.environ.get("FB_PAGE_ID")
token = os.environ.get("FB_PAGE_LONG_TOKEN")

message = "Hi from Jupyter notebook!"
image = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/1200px-Jupyter_logo.svg.png"

# If we want to post a message (no image)
url = f"https://graph.facebook.com/{page_id}/feed?message={message}&access_token={token}"

# If we want to post a photo (post + image or only image)
url = f"https://graph.facebook.com/{page_id}/photos?url={image}&message={message}&access_token={token}"


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
response = requests.post(url)

In [15]:
response.text

'{"id":"765113262281255","post_id":"108938007630581_765113275614587"}'

![fbpost](../images/fb_post.png)

#### Comment Management
To manage comments in automatic we need some way to keep track of all our posts and all the comments under those.

As comments in Facebook assume a tree/nested structure we also need some way to go through the structure and retrieve them.

Then we can use our OpenAI model to create the reply for each message.

In [16]:
from db.db_manager import get_post_from_id, get_social_from_id, full_business_info_as_text, get_business_from_id, \
update_social_post, insert_df_into_table

def get_post_comments(post_id):

    # get the post, social and business from our db
    post = get_post_from_id(post_id)
    social = get_social_from_id(post.social_id)
    business = get_business_from_id(social.business_id)
    token = social.page_token

    # we store the comments we have already managed in a string column in our post table
    if post.managed_comments is None:
        managed_comments = []
    else:
        managed_comments = eval(post.managed_comments)

    # create a business description for the chatbot context
    business_info = full_business_info_as_text(
        social.business_id,
        ["name", "founder_info", "description", "products_and_services_detailed", "competitive_advantage"])

    # initialize variables
    comments_data = []
    comment_history = ""

    # this is a recursive method we are going to call to traverse all the comments
    managed_comments, comments_data = manage_comments_recursive(
        business_info, post, social.remote_id + "_" + post.remote_id, social.remote_id, token, comment_history,
        managed_comments, comments_data)

    return managed_comments, comments_data

In [17]:
import re
from llm_manager.openai_api import openai_request_sync

def manage_comments_recursive(business_info, post, object_id, fb_page_id, token, comment_history_start,
                              managed_comments, comments_data):
    
    headline = post.headline
    content = post.content
    article = post.article

    # get comments for the current node
    url = f"https://graph.facebook.com/{object_id}/comments?access_token={token}"

    response = requests.get(url)
    data = response.json()

    # for each comment
    for c in data["data"]:

        # comment text
        comment = c["message"]

        # remove words starting with @
        comment = re.sub(r'@\w+(\.\w+)*\s?', '', comment)

        # comment_id, to see if we already managed this comment
        comment_id = c["id"]

        # is a comment from a user or from us?
        user_comment = c["from"]["id"] != fb_page_id
        commenter = "user" if user_comment else "business"

        # build comment history context while we traverse the tree
        comment_history = comment_history_start + f"\n- {commenter}: {comment}"

        if user_comment and comment_id not in managed_comments:

            language = "English"

            # get OpenAI model prompt
            with open(os.path.join(os.path.abspath(os.path.join('..')), "prompt\\manage_comments_recursive"), 'r') as file:
                prompt = eval(file.read())

            # call OpenAI API
            response, tokens = openai_request_sync(prompt)

            reply = response["reply"]

            comments_data.append({
                "post_id": post.id,
                "remote_id": comment_id,
                "sender": str(c["from"]),
                "receive_time": c["created_time"],
                "content": comment,
                "reply": reply,
            })

            managed_comments.append(comment_id)

        # call the same method recursively on the comment to explore its comments (if any)
        managed_comments, comments_data = manage_comments_recursive(
            business_info, post, comment_id, fb_page_id, token, comment_history, managed_comments, comments_data)

    return managed_comments, comments_data

In [18]:
managed_comments, comments_data = get_post_comments(1)

In [19]:
for comment in comments_data:
    print(f"{comment['content']} => {comment['reply']}\n")

prova commento mistery => Thank you for your comment! If you have any questions or need any gardening tips, feel free to ask. We're here to help!

Prova commento => Thank you for your comment! We're glad you're interested in learning the secrets of successful cultivation. Ezio, our expert horticulturist, will be happy to share his knowledge and techniques with you. Stay tuned for more updates!

Logiko.io prova commento innestato => Thank you for your comment! If you have any questions about our products or gardening tips, feel free to ask. We're here to help!

Prova commento 2 => Thank you for your comment! If you have any questions about our products or gardening tips, feel free to ask. We're here to help!

Logiko.io chi è Ezio? => Ezio è il fondatore di Le Piantine di Ezio, un esperto orticoltore con una grande passione per l'orticultura. Ha trascorso tutta la sua vita coltivando frutta, verdura e piantine nel suo orto. Ora, in pensione all'età di 67 anni, desidera condividere i prod

We can then store the replies, review/edit them and send them!

<a id='email'></a>
### Email Management

We also created an Email management module (currently not available on our frontend) to be able to programmatically retrieve received emails and manage users' queries.

Like for the Social Management module, the idea is to automatize the customer support via email.

In [20]:
import imaplib, email

from email_manager import process_email_part, process_plain_text_part

def retrieve_emails_from_server(num_emails=3):

    load_dotenv()
    imap_server = os.environ.get("EMAIL_IMAP_SERVER")
    username = os.environ.get("EMAIL_USERNAME")
    password = os.environ.get("EMAIL_PASSWORD")

    imap = None

    try:
        imap = imaplib.IMAP4_SSL(imap_server)
        imap.login(username, password)
        status, messages = imap.select("INBOX")
        messages = int(messages[0])

        result = []

        start_remote_id = messages - num_emails

        for i in range(start_remote_id + 1, messages + 1):

            d = {}

            # fetch the message by ID
            res, msg = imap.fetch(str(i), "(RFC822)")

            # parse the message into an email_manager object
            msg = email.message_from_bytes(msg[0][1]) # sync

            # retrieve email metadata
            d["sender"] = msg["From"]
            date = msg["Date"] if msg["Date"] is not None else msg["Delivery-date"]
            d["receive_time"] = date
            d["subject"] = msg['Subject']
            d["recipient"] = ";".join(msg.get_all('To', []))
            d["cc"] = ";".join(msg.get_all('Cc', []))
            d["remote_id"] = i

            # retrieve email content (we want to extract the text from the multipart)
            for part in msg.walk():
                part_content_raw, part_content_clean = process_plain_text_part(part)
                if part_content_raw is not None:
                    d["content"] = part_content_clean
                    d["content_raw"] = part_content_raw
                    break
                else:
                    part_content_raw, part_content_clean = process_email_part(part)
                    if part_content_raw is not None:
                        d["content"] = part_content_clean
                        d["content_raw"] = part_content_raw
                        break

            result.append(d)
    finally:
        if imap is not None:
            imap.logout()

    return result

In [21]:
emails = retrieve_emails_from_server()
emails

[{'sender': 'mistery@easymap-software.com',
  'receive_time': 'Fri, 04 Aug 2023 16:02:59 +0000',
  'subject': 'Inquiry about Fresh, Locally Sourced Produce',
  'recipient': 'lepiantinediezio@easymap-software.com',
  'cc': '',
  'remote_id': 12,
  'content': 'From: Emily Johnson To: marketing@company.com Dear Sir/Madam, I hope this email finds you well. My name is Emily Johnson, and I recently came across an article on your company blog titled "Discover the Delights of Fresh, Locally Sourced Produce." I was intrigued by the information provided and would like to learn more about your offerings. The article mentioned the advantages of consuming fresh, locally sourced produce, such as supporting local agriculture, enjoying superior taste and quality, and the positive impact on the environment. I am particularly interested in understanding how your company ensures the freshness and quality of the produce you offer. Could you please provide me with more information about your sourcing proce

In [22]:
def write_email_replies(emails, business_id):

    business = get_business_from_id(business_id)
    business_info = full_business_info_as_text(
        business_id,
        ["name", "founder_info", "description", "competitive_advantage", "products_and_services_detailed"])

    signature = business.name
    language = "English"

    for em in emails:
        sender = em["sender"]
        subject = em["subject"]
        content = em["content"]

        # get OpenAI model prompt
        with open(os.path.join(os.path.abspath(os.path.join('..')), "prompt\\write_email_replies"), 'r') as file:
            prompt = eval(file.read())

        # call OpenAI API
        response, tokens = openai_request_sync(prompt, context="long")
        
        em["reply"] = response["reply"]

    return emails

In [23]:
emails_replies = write_email_replies(emails, 3535)

In [24]:
for em in emails_replies:
    print(f"{em['content']} \n\n\t ### REPLY ### \n\n {em['reply']}\n\n=================\n")

From: Emily Johnson To: marketing@company.com Dear Sir/Madam, I hope this email finds you well. My name is Emily Johnson, and I recently came across an article on your company blog titled "Discover the Delights of Fresh, Locally Sourced Produce." I was intrigued by the information provided and would like to learn more about your offerings. The article mentioned the advantages of consuming fresh, locally sourced produce, such as supporting local agriculture, enjoying superior taste and quality, and the positive impact on the environment. I am particularly interested in understanding how your company ensures the freshness and quality of the produce you offer. Could you please provide me with more information about your sourcing process? How do you select the local farmers you work with? Do you have any certifications or quality control measures in place to guarantee the freshness and nutritional value of the produce? Additionally, I would like to know if you have any upcoming events, suc

Like for the comments we can now store the replies, review/edit them and send them!

## Conclusion

Visit https://smark-frontend.vercel.app to access the Marketing Automation frontend
and unleash the power of AI-driven marketing.

For any inquiries or feedback, please reach out to our team at [blacklionsrl.italia@gmail.com](mailto://blacklionsrl.italia@gmail.com)